In [ ]:
import datetime as dt
import praw
from psaw import PushshiftAPI

In [ ]:
reddit = praw.Reddit(
    client_id="NEuQE23gPSC5QA",
    client_secret="pVJemwClzr77juCfn3M65vsAYDc",
    user_agent="research",
    username="notNotLurking",
    password="Bebrave!Havefun!",
)
api = PushshiftAPI(reddit)

Grab comments... this can take a while... like 15s/1k comments

In [ ]:
comments = list(api.search_comments(subreddit="politics", limit=10_000))

print("We grabbed %s comments" % len(comments))

Let's figure out which attributes each exist on at least one of the comments, and are not private.

In [ ]:
attributes = set()
for s in comments:
    attributes.update(vars(s).keys())
public_attributes = sorted(s for s in attributes if s[0] != "_")

How many are there?

In [ ]:
len(public_attributes)

Let's see how many times each one that doesn't appear every time appears

In [ ]:
from collections import Counter

counter = Counter()
for s in comments:
    counter.update(vars(s).keys())
sorted(
    (t for t in counter.items() if t[0] in public_attributes and t[1] < len(comments)),
    key=lambda t: t[1],
)

In [ ]:
len(_)

Let's see which attributes never appear with any value other than None

In [ ]:
def none_only_attribute(attribute):
    for s in comments:
        if attribute in vars(s).keys() and getattr(s, attribute) != None:
            return False
    return True


none_only_attributes = [a for a in public_attributes if none_only_attribute(a)]
none_only_attributes

In [ ]:
len(none_only_attributes)

Which attributes are not always present, but when present, always have the value None? (This may be an empty set.)

In [ ]:
set(none_only_attributes) - set(
    a for a in none_only_attributes if all(hasattr(s, a) for s in comments)
)

In [ ]:
def value_for_attribute(attribute):
    """Try to find a non-None value for the given attribute"""
    for s in comments:
        if attribute in vars(s).keys() and getattr(s, attribute) != None:
            return getattr(s, attribute)
    return None

In [ ]:
def value_for_attribute_nonfalsy(attribute):
    """Try to find a non-Falsy value for the given attribute"""
    for s in comments:
        if attribute in vars(s).keys() and getattr(s, attribute):
            return getattr(s, attribute)
    return value_for_attribute(attribute)

Which types are taken on by the values of all the public attributes?

In [ ]:
set(type(value_for_attribute(a)) for a in public_attributes)

In [ ]:
def attributes_for_type(t):
    return [a for a in public_attributes if type(value_for_attribute(a)) == t]

Note the author and subreddit attributes are praw things computed from author_fullname and subreddit_id

Which attributes take on dict values?

In [ ]:
dict_attributes = attributes_for_type(dict)
dict_attributes

Which attributes take on list values?

In [ ]:
list_attributes = attributes_for_type(list)
list_attributes

This code can be used to "sample" interesting values for attributes. It skips Nones and tries to skip empty collections

In [ ]:
a = "author_flair_richtext"
v = value_for_attribute_nonfalsy(a)
v if v else value_for_attribute(a)

This code computes up all (optionally, non-falsy) values taken on by the attribute

In [ ]:
non_falsy = True
[
    getattr(s, a)
    for s in comments
    if a in vars(s).keys() and (not non_falsy or getattr(s, a))
]

In [ ]:
primitives = [bool, float, int, str]

In [ ]:
primitive_attributes = [
    a for a in public_attributes if type(value_for_attribute(a)) in primitives
]
[(a, type(value_for_attribute_nonfalsy(a))) for a in primitive_attributes]

In [ ]:
def longest_str(attribute):
    return max(
        len(getattr(s, attribute))
        for s in comments
        if attribute in vars(s).keys() and getattr(s, attribute)
    )

In [ ]:
attribute_lengths = {}
for a in primitive_attributes:
    if type(value_for_attribute(a)) == str:
        attribute_lengths[a] = longest_str(a)
attribute_lengths